In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import random
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,Input,BatchNormalization,MaxPooling1D,concatenate
from keras.utils import np_utils,multi_gpu_model
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from sklearn.cluster import KMeans
from scipy import stats
from sklearn import preprocessing
from keras import backend as bek
import gc
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
"""
data cell
"""
df1=pd.read_csv('/home/dp/Desktop/1_12_train_BG/pair_end_merge_BG_random_HR-train_12.csv',index_col=0)


cond2 = df1['count']!=0
df1 = df1[cond2]

#normalize
target = 'BG'

In [ ]:
def c(ref_values, pred_values):

    #Checking to see if the lengths of the reference and prediction arrays are the same
    assert (len(ref_values) == len(pred_values)), "Unequal number of values (reference : {}) (prediction : {}).".format(len(ref_values), len(pred_values))

    #Checks to see if the values are within the normal physiological range, otherwise it gives a warning
    if max(ref_values) > 400 or max(pred_values) > 400:
        print("Input Warning: the maximum reference value {} or the maximum prediction value {} exceeds the normal physiological range of glucose (<400 mg/dl).".format(max(ref_values), max(pred_values)))
    if min(ref_values) < 0 or min(pred_values) < 0:
        print("Input Warning: the minimum reference value {} or the minimum prediction value {} is less than 0 mg/dl.".format(min(ref_values),  min(pred_values)))

    #Statistics from the data
    zone = [0] * 5
    for i in range(len(ref_values)):
        if (ref_values[i] <= 70 and pred_values[i] <= 70) or (pred_values[i] <= 1.2*ref_values[i] and pred_values[i] >= 0.8*ref_values[i]):
            zone[0] += 1    #Zone A

        elif (ref_values[i] >= 180 and pred_values[i] <= 70) or (ref_values[i] <= 70 and pred_values[i] >= 180):
            zone[4] += 1    #Zone E

        elif ((ref_values[i] >= 70 and ref_values[i] <= 290) and pred_values[i] >= ref_values[i] + 110) or ((ref_values[i] >= 130 and ref_values[i] <= 180) and (pred_values[i] <= (7/5)*ref_values[i] - 182)):
            zone[2] += 1    #Zone C
        elif (ref_values[i] >= 240 and (pred_values[i] >= 70 and pred_values[i] <= 180)) or (ref_values[i] <= 175/3 and pred_values[i] <= 180 and pred_values[i] >= 70) or ((ref_values[i] >= 175/3 and ref_values[i] <= 70) and pred_values[i] >= (6/5)*ref_values[i]):
            zone[3] += 1    #Zone D
        else:
            zone[1] += 1    #Zone B
            
    confidence_score = round((zone[0]*5+zone[1]*2.5)/(len(ref_values)*5)*100,2)
    
    return confidence_score,zone

In [ ]:
def c1(ref_values, pred_values):
    
    sum_a = 0
    for i in range(len(ref_values)):
        d = abs(pred_values[i]-ref_values[i])
        if((d/ref_values[i])<1):
            a = abs(d/ref_values[i]-1)
        else:
            a = 0
        sum_a = sum_a + a
    acc = sum_a/len(ref_values)
    acc = round(acc*100,2)
    return acc

In [ ]:
'''
model cell
'''
def base_model():
    in_s = Input((406, 2))
    model_s = Convolution1D(nb_filter=256, filter_length=3) (in_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=256, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=512, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=1024, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=2048, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Flatten()(model_s)
    
    input1 = keras.layers.Input(shape=(1,))
    c = keras.layers.Concatenate(axis=-1)([model_s, input1])
    
    m = BatchNormalization(axis=-1)(c)
    m = Dense(4096,activation='relu')(m)
    m = Dense(2048,activation='relu')(m)
    out = Dense(1,activation='linear')(m)
    
    model = Model(inputs=[in_s,input1], outputs=[out])
    
    adam=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999)
    model = multi_gpu_model(model,gpus=6)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
def vali_step(train_temp,vali_temp):
    X_train_s = train_temp.iloc[:,7:413]
    X_train_s_last = train_temp.iloc[:,414:820]
    X_vali_s = vali_temp.iloc[:,7:413]
    X_vali_s_last = vali_temp.iloc[:,414:820]
    
    f1=['BG']
    Y_train = train_temp[f1]
    Y_train = scalert.transform(Y_train)
    
    # reshape train data
    X_train_r_s = np.zeros((len(X_train_s), 406, 2))
    X_train_r_s[:, :, 0] = X_train_s
    X_train_r_s[:, :, 1] = X_train_s_last

    # reshape vali data
    X_vali_r_s = np.zeros((len(X_vali_s), 406, 2))
    X_vali_r_s[:, :, 0] = X_vali_s
    X_vali_r_s[:, :, 1] = X_vali_s_last
    
    
    f2=['BG_last']
    # reshape train data
    X_train_r_f = train_temp[f2]

    # reshape vali data
    X_vali_r_f = vali_temp[f2]
    
    
    #fit
    model= base_model()
    model.fit([X_train_r_s,X_train_r_f], Y_train, epochs=1000, batch_size=3000,verbose=0)

    
    vali_pred = model.predict([X_vali_r_s,X_vali_r_f])
    vali_pred1 = scalert.inverse_transform(vali_pred)
    vali_temp['BG_p']=vali_pred1
    temp_a=0
    temp_b=0
    vali_BG_r=[]
    vali_BG_p=[]
                        
    for i,item in enumerate(vali_temp['Person No']):
        if((vali_temp['Person No'].iloc[i]!=temp_a) | (vali_temp['time'].iloc[i]!=temp_b)):
            
            #administrator mode
            vali_BG_r.append(vali_temp['BG'].iloc[i])
            temp = vali_temp[(vali_temp['Person No']==vali_temp['Person No'].iloc[i])&(vali_temp['time']==vali_temp['time'].iloc[i])]
            t = np.median(temp['BG_p'])
            vali_BG_p.append(t)
            temp_a = vali_temp['Person No'].iloc[i]
            temp_b = vali_temp['time'].iloc[i]
            
    print(vali_BG_p)
    print(vali_BG_r)
    del model
    bek.clear_session()
    gc.collect()
    return (vali_BG_p,vali_BG_r)

In [ ]:
def test_step(train_temp,test_temp):
    X_train_s = train_temp.iloc[:,7:413]
    X_train_s_last = train_temp.iloc[:,414:820]
    X_test_s = test_temp.iloc[:,7:413]
    X_test_s_last = test_temp.iloc[:,414:820]
    
    f1=['BG']
    Y_train = train_temp[f1]
    Y_train = scalert.transform(Y_train)
    
    # reshape train data
    X_train_r_s = np.zeros((len(X_train_s), 406, 2))
    X_train_r_s[:, :, 0] = X_train_s
    X_train_r_s[:, :, 1] = X_train_s_last

    # reshape vali data
    X_test_r_s = np.zeros((len(X_test_s), 406, 2))
    X_test_r_s[:, :, 0] = X_test_s
    X_test_r_s[:, :, 1] = X_test_s_last
    
    f2=['BG_last']
    # reshape train data
    X_train_r_f = train_temp[f2]

    # reshape vali data
    X_test_r_f = test_temp[f2]
    
    #fit
    model= base_model()
    model.fit([X_train_r_s,X_train_r_f], Y_train, epochs=1000, batch_size=3000,verbose=0)

    
    test_pred = model.predict([X_test_r_s,X_test_r_f])
    test_pred1 = scalert.inverse_transform(test_pred)
    test_temp['BG_p']=test_pred1
    temp_a=0
    temp_b=0
    test_BG_r=[]
    test_BG_p=[]
                        
    for i,item in enumerate(test_temp['Person No']):
        if((test_temp['Person No'].iloc[i]!=temp_a) | (test_temp['time'].iloc[i]!=temp_b)):
            
            #administrator mode
            test_BG_r.append(test_temp['BG'].iloc[i])
            temp = test_temp[(test_temp['Person No']==test_temp['Person No'].iloc[i])&(test_temp['time']==test_temp['time'].iloc[i])]
            t = np.median(temp['BG_p'])
            test_BG_p.append(t)
            temp_a = test_temp['Person No'].iloc[i]
            temp_b = test_temp['time'].iloc[i]
            
    print(test_BG_p)
    print(test_BG_r)
    del model
    bek.clear_session()
    gc.collect()
    return (test_BG_p,test_BG_r)

In [ ]:
################
testingNo = 1
################


test_p=[]
test_r=[]
vali_p=[]
vali_r=[]
vali_score=[]
vali_zone=[]
test_zone=[]
acc=[]
score_1=[]
score_2=[]

for test_time in range(13,16):
    test  = df1[(df1['count']==testingNo)&(df1['count_time']==test_time)]
    if(test.empty):
        continue
    else:
        test_index = test.index
        data_base = df1.drop(test_index)
        t = data_base[target]
        t = t.values.reshape(-1, 1)
        scalert = preprocessing.MinMaxScaler().fit(t)
        test_train = data_base[(data_base['count']==testingNo)&(data_base['count_time']<13)]
    
        for vali_time in range(2,13):
            vali  = data_base[(data_base['count']==testingNo)&(data_base['count_time']==vali_time)]
            if(vali.empty):
                continue
            else:
                vali_index = vali.index
                vali_train = data_base[(data_base['count']==testingNo)&(data_base['count_time']<13)]
                vali_train = vali_train.drop(vali_index)
                (temp_pred,temp_real) = vali_step(vali_train,vali)
                print("test No:",testingNo," test time:",test_time," vali time:",vali_time)
                vali_p.append(temp_pred)
                vali_r.append(temp_real)
        
        reshape_p=[]
        reshape_r=[]

        for i in range(len(vali_p)):
            for j in range(len((vali_p[i]))):
                reshape_p.append(vali_p[i][j])
        for i in range(len(vali_r)):
            for j in range(len((vali_r[i]))):
                reshape_r.append(vali_r[i][j]) 
        vali_p=[]
        vali_r=[]        
        vali_conf,vali_z = c(reshape_r,reshape_p)
        vali_score.append(vali_conf)
        vali_zone.append(vali_z)
        
        p_1=[]
        p_2=[]
        for i in range(0,10):
            (temp_pred,temp_real) = test_step(test_train,test)
            p_1.append(temp_pred[0])
            p_2.append(temp_pred[1])
        p_1.remove(max(p_1))
        p_1.remove(min(p_1))
        p_2.remove(max(p_2))
        p_2.remove(min(p_2))
        s_1 = np.std(p_1)/np.median(p_1)
        s_2 = np.std(p_2)/np.median(p_2)
        score_1.append(s_1)
        score_2.append(s_2)
        
        
        m_pred=[np.median(p_1),np.median(p_2)]
        
        test_p.append(m_pred)
        test_r.append(temp_real)
        
        reshape_p=[]
        reshape_r=[]

        for i in range(len(m_pred)):
                reshape_p.append(m_pred[i])
        for i in range(len(temp_real)):
                reshape_r.append(temp_real[i])
                
        no_mind,test_z = c(reshape_r,reshape_p)
        test_acc = c1(reshape_r,reshape_p)
        
        acc.append(test_acc)
        test_zone.append(test_z)
        
        print("test No:",testingNo,"test time:",test_time,"confidence score:",vali_conf,"%",'vali_zone:',vali_z)
        print("BG real:",temp_real," BG pred:",m_pred,"accuracy:",test_acc,"%",'test_zone:',test_z)

In [ ]:
d = {'confidence score':vali_score, 'vali_zone':vali_zone , 'BG_real': test_r, 'BG_pred': test_p, 'test_zone':test_zone,'score_1':score_1,'score_2':score_2,'accuracy':acc}
df_f = pd.DataFrame(data = d)
df_f

In [ ]:
ind =[]
for j in range(13,len(df_f.index)+13):
    ind.append(j)
    
df_f.index=ind

In [ ]:
#screen
mark = []
reason = []

for i,item in enumerate(df_f['confidence score']):
    print(i)
    his_M = (max(set(df1[(df1['count']==testingNo)&(df1['count_time'] < df_f.index[i])]['BG'])))
    his_m = (min(set(df1[(df1['count']==testingNo)&(df1['count_time'] < df_f.index[i])]['BG'])))
    #print(his_M,his_m)
    if((df_f.index[i]==4)|(df_f.index[i]==5)):
        if(df_f['confidence score'].iloc[i] < 50):
            print('test time :',df_f.index[i],' reason 1')
            mark.append('reject')
            reason.append(1)
        else:
            if( (df_f['score_1'].iloc[i]>0.07) | (df_f['score_2'].iloc[i] > 0.07) ):
                print('test time :',df_f.index[i],' reason 2')
                mark.append('reject')
                reason.append(2)
            else:
                temp=[]
                for j in range(len([float(item) for item in df_f['BG_pred'].iloc[i][1:-1].split(',')])):
                    temp.append([float(item) for item in df_f.iloc[i]['BG_pred'][1:-1].split(',')][j])
                thistime_M=max(temp)
                thistime_m=min(temp)
                if( (thistime_M > (his_M*1.1)) | ((thistime_m < (his_m*0.9)) )):
                    print('test time :',df_f.index[i],' reason 3')
                    mark.append('reject')
                    reason.append(3)
                else:
                    if(thistime_m < 200):
                        if(thistime_m*1.3 < thistime_M):
                            print('test time :',df_f.index[i],' reason 4')
                            mark.append('reject')
                            reason.append(4)
                        else:
                            print('test time :',df_f.index[i],' pass')
                            mark.append('pass')
                            reason.append(0)
                    else:
                        if(thistime_m*1.35 < thistime_M):
                            print('test time :',df_f.index[i],' reason 4')
                            mark.append('reject')
                            reason.append(4)
                        else:
                            print('test time :',df_f.index[i],' pass')
                            mark.append('pass')
                            reason.append(0)
    else:
        if(df_f['confidence score'].iloc[i] < 60):
            print('test time :',df_f.index[i],' reason 1')
            mark.append('reject')
            reason.append(1)
        else:
            if( (df_f['score_1'].iloc[i]>0.07) | (df_f['score_2'].iloc[i] > 0.07) ):
                print('test time :',df_f.index[i],' reason 2')
                mark.append('reject')
                reason.append(2)
            else:
                temp=[]
                for j in range(len([float(item) for item in df_f['BG_pred'].iloc[i][1:-1].split(',')])):
                    temp.append([float(item) for item in df_f.iloc[i]['BG_pred'][1:-1].split(',')][j])
                thistime_M=max(temp)
                thistime_m=min(temp)
                if( (thistime_M > (his_M*1.1)) | ((thistime_m < (his_m*0.9)) )):
                    print('test time :',df_f.index[i],' reason 3')
                    mark.append('reject')
                    reason.append(3)
                else:
                    if(thistime_m < 200):
                        if(thistime_m*1.3 < thistime_M):
                            print('test time :',df_f.index[i],' reason 4')
                            mark.append('reject')
                            reason.append(4)
                        else:
                            print('test time :',df_f.index[i],' pass')
                            mark.append('pass')
                            reason.append(0)
                    else:
                        if(thistime_m*1.35 < thistime_M):
                            print('test time :',df_f.index[i],' reason 4')
                            mark.append('reject')
                            reason.append(4)
                        else:
                            print('test time :',df_f.index[i],' pass')
                            mark.append('pass')
                            reason.append(0)

In [ ]:
df_f['mark']=mark
df_f